In [1]:
import tweepy
import pandas as pd
from textblob import TextBlob
import nltk
import csv

In [2]:
%run ~/twitter_credentials.py

In [3]:

#Use tweepy.OAuthHandler to ```create an authentication using the given key and secret
auth = tweepy.OAuthHandler(consumer_key=con_key, consumer_secret=con_secret)
auth.set_access_token(acc_token, acc_secret)

#Connect to the Twitter API using the authentication
api = tweepy.API(auth)

In [4]:
class MyStreamListener(tweepy.StreamListener):

    def __init__(self):
        
        super(MyStreamListener, self).__init__()
        self.num_tweets = 0
        self.tweet_list = []
        self.rt_status = 'RT @'
        self.retweet_id = ''

    def on_status(self, status):
        if self.num_tweets < 1000:
            if self.rt_status in status.text:# can also retweet using RT @ and doesn't show up as retweet
                tweet_is_Retweet = True
                self.retweet_id = status.text.split('RT')[1].split(':')[0]
            else:
                tweet_is_Retweet = False
                self.retweet_id = ''
                
            if hasattr(status, 'retweeted_status'):
                try:
                    tweet = status.retweeted_status.extended_tweet["full_text"]
                               
                except:
                    tweet = status.retweeted_status.text
                     
            else:
                try:
                    tweet = status.extended_tweet["full_text"]
                     
                except AttributeError:
                    tweet = status.text
                     
    
            #tweet = status.text
            self.num_tweets  += 1
            tweet_len = 0       
            analysis = TextBlob(tweet)
            subjectivity =  analysis.sentiment.subjectivity
            polarity = analysis.sentiment.polarity 
            screen_name = status.user.screen_name   
            retweet_at = self.retweet_id
            retweet_count = status.retweet_count
            tweet_len = len(tweet)
            in_reply_to_screen_name = status.in_reply_to_screen_name
            full_tweet_list = [screen_name, tweet, polarity,subjectivity,retweet_count,retweet_at,in_reply_to_screen_name,tweet_len,tweet_is_Retweet]
            self.tweet_list.append(full_tweet_list)
             
        else:
            #tweet_df = pd.DataFrame(columns=['screen_name', 'tweet', 'polarity','subjectivity','retweet_count','retweet_at',in_reply_to_screen_name','tweet_len','tweet_is_Retweet'])
            tweet_df = pd.DataFrame(self.tweet_list,columns=['screen_name', 'tweet', 'polarity','subjectivity','retweet_count','retweet_at','in_reply_to_screen_name','tweet_len','tweet_is_Retweet'])
            tweet_df.to_csv('Brexit_tweets.csv',index = False)
            myStream.disconnect()
            return
            
    def on_error(self, status_code):
            print ('Encountered error with status code:', status_code)
            return True # Don't kill the stream

    def on_timeout(self):
        tweet_df = pd.DataFrame(self.tweet_list,columns=['screen_name', 'tweet', 'polarity','subjectivity','retweet_count','retweet_at','in_reply_to_screen_name','tweet_len','tweet_is_Retweet'])
        tweet_df.to_csv('Brexit_tweets.csv',index = False)
        print('Timeout...')
        return True # Don't kill the stream


In [5]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

In [6]:
myStream.filter(track=['#Brexit'], is_async=True,languages=["en"])